In [1]:
'''
 Note: The dictionary for this program is prepared using the words in the file sonnets.txt.
 Hence this file is required for this program to work.
 Imported libr
'''

# libraries
import re
import string
from collections import Counter
import numpy as np
from tkinter import *
import tkinter as tk
from tkinter import messagebox


# -- Prepare base window
root = Tk()
root.title("PYTHON: AUTO-CORRECTION FORM")
root.geometry("700x500")
root.config(bg='#f0f0f0')



# -- Function to call on the envent OnClick
def onClick():
    checker = auto_correct("./sonnets.txt")
    result = checker.check(w_value.get())
    messagebox.showinfo(title="Auto-Correct Result", message=result)

# -- Class starts here

class auto_correct(object):
    def __init__(self, corpus_file_path):
        with open(corpus_file_path, "r") as file:
            lines = file.readlines()
            words = []
        for line in lines:
            words += re.findall(r'\w+', line.lower())
        self.vocabs = set(words)# as we need unique words
        self.word_counts = Counter(words)
        total_words = float(sum(self.word_counts.values()))
        self.word_probas = {word: self.word_counts[word] / total_words for word in self.vocabs}

    def _level_one_edits(self, word):
        #we need words in english and in lowercase.
        letters = string.ascii_lowercase
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [l + r[1:] for l,r in splits if r]
        swaps = [l + r[1] + r[0] + r[2:] for l, r in splits if len(r)>1]
        replaces = [l + c + r[1:] for l, r in splits if r for c in letters]
        inserts = [l + c + r for l, r in splits for c in letters] 
        return set(deletes + swaps + replaces + inserts)

    def _level_two_edits(self, word):
        return set(e2 for e1 in self._level_one_edits(word) for e2 in self._level_one_edits(e1))

    def check(self, word):
        candidates = self._level_one_edits(word) or self._level_two_edits(word) or [word]
        valid_candidates = [w for w in candidates if w in self.vocabs]
        return sorted([(c, self.word_probas[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)


    
    
img = PhotoImage(file="picku.png",master=root)
labela = Label(root,image=img)
labela.place(x=1050, y=300)
w_value = StringVar()
# Adding a label into window
label1= Label(root,text ="AUTO-CORRECT SYSTEM",bg= "black",pady=15, padx=20,fg ="white",font = ("Arial",50,'bold'),borderwidth = 10,relief = 'groove')
label1.pack()
label2 = Label(root,text ="Welcome!",pady=15, padx=20,fg ="black",font = ("Arial",30,'underline'))
label2.pack()
word_label = Label(root,text = "Enter the word: ",pady=5, padx=1,fg ="black",font = ("Arial",15,'underline'))
word_label.pack(anchor = 'w')

#Adding an entry box
word_entry = Entry(root,textvariable=w_value,font = ("Arial",12),bg= "darkgrey",bd=5)
word_entry.pack(anchor = 'w')

#Adding a submit button
B1 = Button(root,text = "Submit",relief="raised",font = ("Arial",10),command = onClick)
B1.pack(anchor = 'w')
B1.place(x = 2, y = 280)
root.mainloop()